# Problema 4.3

Problema de Transporte

- Uma empresa possui `3` unidades industriais em diferentes regiões e coloca o seu produto em `4` centros consumidores.
- Existe um equilíbrio entre a oferta e a demanda.

In [11]:
ofertas = {
    "fabrica1": 17,
    "fabrica2": 12,
    "fabrica3": 10,
}

demandas = {
    "b1": 9,  # centro de distribuição 1
    "b2": 5,  # centro de distribuição 2
    "b3": 8,  # centro de distribuição 3
    "b4": 17,  # centro de distribuição 4
}

custo_unit_producao = {
    "fabrica1": 50,
    "fabrica2": 45,
    "fabrica3": 54,
}

preco_revenda_centros = {  # Quanto a empresa ganha por vender em cada CD.
    "b1": 65,
    "b2": 60,
    "b3": 58,
    "b4": 62,
}

custo_transporte = {
    "fabrica1": {
        "b1": 6,
        "b2": 3,
        "b3": 5,
        "b4": 8,
    },
    "fabrica2": {
        "b1": 7,
        "b2": 6,
        "b3": 4,
        "b4": 6,
    },
    "fabrica3": {
        "b1": 3,
        "b2": 5,
        "b3": 3,
        "b4": 4,
    },
}

## Parte 1: 

Determinar a forma em que se processa o transporte entre as unidades industriais e os centros consumidores.

In [12]:
import docplex.mp.model as cpx

mdl = cpx.Model(name="Problema 4.3")

Seja $F$ o conjunto de unidades industriais (fábricas) e $D$ o conjunto de centros consumidores (depósitos).

As variáveis de decisão são do tipo:

$x_{i,j} \in \mathbb{R} \quad \quad \forall i \in F \quad \forall j \in D $


In [13]:
# Variáveis de decisão
## Transporte da fabrica i até o depósito j

for i, fabrica in enumerate(ofertas.keys()):
    for j, deposito in enumerate(demandas.keys()):
        # Já estou garantindo que a variável é não negativa
        mdl.continuous_var(lb=0, ub=mdl.infinity, name=f"x_{i+1}_{j+1}")

# Função objetivo
## Minimizar custo total, que é a soma entre custo de transporte e custo de revenda.

lucro_venda = mdl.sum(
    mdl.sum(
        preco_revenda_centros[deposito] * mdl.get_var_by_name(f"x_{i+1}_{j+1}")
        for j, deposito in enumerate(demandas.keys())
    )
    for i, fabrica in enumerate(ofertas.keys())
)

C_producao = mdl.sum(
    custo_unit_producao[fabrica] * ofertas[fabrica] for fabrica in ofertas.keys()
)

C_transporte = mdl.sum(
    mdl.sum(
        custo_transporte[fabrica][deposito] * mdl.get_var_by_name(f"x_{i+1}_{j+1}")
        for j, deposito in enumerate(demandas.keys())
    )
    for i, fabrica in enumerate(ofertas.keys())
)


resultado = lucro_venda - C_transporte - C_producao

mdl.maximize(resultado)

# Restrições
## As demandas dos centros b1, b2, b3 e b4 devem ser atendidas.

for j, deposito in enumerate(demandas.keys()):
    mdl.add_constraint(
        mdl.sum(
            mdl.get_var_by_name(f"x_{i+1}_{j+1}")
            for i, fabrica in enumerate(ofertas.keys())
        )
        == demandas[deposito]
    )

## As ofertas das fábricas 1, 2 e 3 devem ser respeitadas.

for i, fabrica in enumerate(ofertas.keys()):
    mdl.add_constraint(
        mdl.sum(
            mdl.get_var_by_name(f"x_{i+1}_{j+1}")
            for j, deposito in enumerate(demandas.keys())
        )
        == ofertas[fabrica]
    )

In [14]:
mdl.export("problema_4.3.lp")

'problema_4.3.lp'

In [15]:
solution = mdl.solve()

print(mdl.get_solve_details())
print(f"Solution: {mdl.solve()}")

status  = optimal
time    = 0 s.
problem = LP

Solution: solution for: Problema 4.3
objective: 287
status: OPTIMAL_SOLUTION(2)
x_1_1=9.000
x_1_2=5.000
x_1_3=3.000
x_2_3=5.000
x_2_4=7.000
x_3_4=10.000



## Parte 2:

Este equilíbrio entre oferta e demanda foi conseguido ao longo de um processo de competição pelo mercado.
Admita que um acidente ocorreu na fábrica 1 e a produção cai para 7.

Se a empresa estima que o impacto sobre as vendas futuras pode ser medido através de uma penalidade $p_{j}$, por unidade de demanda não atendida, qual será o novo esquema de distribuição? 
Os valores de $p_{j}$ são 4, 7, 3 e 2, respectivamente.

In [16]:
# TODO: fazer depois